In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F

from torch.utils.data import DataLoader, Dataset

import numpy as np

In [2]:
batch_size = 64
num_epochs = 30
latent_dim = 256
num_samples = 10000
data_path = 'data/fra-eng/fra.txt'

In [3]:
class DataWrapper(Dataset):
    def __init__(self, data_path, num_samples):
        self.data_path = data_path
        
        self.input_texts = []
        self.target_texts = []

        self.input_vocab = set()
        self.output_vocab = set()

        with open(data_path, 'r', encoding='utf-8') as f:
            lines = f.read().split('\n')
        for line in lines[: min(num_samples, len(lines) - 1)]:
            input_text, target_text, _ = line.split('\t')
            target_text = '\t' + target_text + '\n'
            self.input_texts.append(input_text)
            for word in input_text.split():
                self.input_vocab.add(word.strip())
            self.target_texts.append(target_text)
            for word in target_text.split():
                self.output_vocab.add(word.strip())
        
        self.input_vocab2index = {word: i+2 for i, word in enumerate(self.input_vocab)}
        self.output_vocab2index = {word: i+2 for i, word in enumerate(self.output_vocab)}
        
        self.training_pairs = np.random.randint(0, len(self.input_texts), size=num_samples)

    def indexesFromSentence(self, sentence, vocab):
        return [vocab.get(word.strip(), 0) for word in sentence.split(' ')]

    def tensorFromSentence(self, sentence, vocab):
        indexes = self.indexesFromSentence(sentence, vocab)
        indexes.append(1)
        return torch.tensor(indexes, dtype=torch.long).view(-1, 1)

    def tensorsFromSent(self, input_sentences, output_sentences):
        input_tensor = self.tensorFromSentence(input_sentences, self.input_vocab2index)
        target_tensor = self.tensorFromSentence(output_sentences, self.output_vocab2index)
        return (input_tensor, target_tensor)

    def __getitem__(self, index):
        return self.tensorsFromSent(self.input_texts[self.training_pairs[index]],
                               self.target_texts[self.training_pairs[index]])
    
    def __len__(self):
        return len(self.input_texts)

In [4]:
# Based on: https://discuss.pytorch.org/t/dataloader-for-various-length-of-data/6418/7
class PadCollate:
    """
    a variant of collate_fn that pads according to the longest sequence in
    a batch of sequences
    """

    def __init__(self):
        pass

    def pad_collate(self, batch):
        """
        args:
            batch - list of [input, target]
        return:
            inputs - a tensor of all inputs in batch after padding
            targets - a tensor of all targets in batch after padding
        """

        # Find longest sequence
        max_input_length = max([len(input_target_pair[0]) for input_target_pair in batch])
        max_target_length = max([len(input_target_pair[1]) for input_target_pair in batch])

        # Pad 'm
        batch = [[self.pad_tensor(input_target_pair[0], max_input_length, pad_front=True), \
                  self.pad_tensor(input_target_pair[1], max_target_length, pad_front=False)] \
                 for input_target_pair in batch]

        # Stack the inputs together and the targets together
        inputs = torch.stack([input_target_pair[0] for input_target_pair in batch])
        targets = torch.stack([input_target_pair[1] for input_target_pair in batch])
        
        return inputs, targets

    def pad_tensor(self, vec, pad, pad_front):
        """
        args:
            vec - tensor to pad
            pad - the size to pad to
        return:
            a new tensor padded to 'pad' in dimension 'dim'
        """
        pad_size = list(vec.shape)
        pad_size[0] = pad - vec.size(0)

        vec = vec.type(torch.LongTensor)

        if pad_front:
            return torch.cat([torch.zeros(*pad_size).type(torch.LongTensor), vec], dim=0)
        else:
            return torch.cat([vec, torch.zeros(*pad_size).type(torch.LongTensor)], dim=0)


    def __call__(self, batch):
        return self.pad_collate(batch)

In [5]:
data_set = DataWrapper(data_path, num_samples)
data_loader = DataLoader(data_set, batch_size=batch_size, shuffle=True, num_workers=0,
                            collate_fn=PadCollate())

In [6]:
for i, (data, target) in enumerate(data_loader):
    print(len(data))
    print(data.size())
    print(target.size())
    break

64
torch.Size([64, 5, 1])
torch.Size([64, 8, 1])


In [7]:
class EncoderRNN(nn.Module):
    def __init__(self, input_size, hidden_size):
        super(EncoderRNN, self).__init__()
        self.hidden_size = hidden_size

        self.embedding = nn.Embedding(input_size, hidden_size)
        self.gru = nn.GRU(hidden_size, hidden_size)

    def forward(self, input_, hidden):
        embedded = self.embedding(input_).view(1, 1, -1)
        output = embedded
        output, hidden = self.gru(output, hidden)
        return output, hidden

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)


class AttnDecoderRNN(nn.Module):
    def __init__(self, hidden_size, output_size, dropout_p=0.1, max_length=10):
        super(AttnDecoderRNN, self).__init__()
        self.hidden_size = hidden_size
        self.output_size = output_size
        self.dropout_p = dropout_p
        self.max_length = max_length

        self.embedding = nn.Embedding(self.output_size, self.hidden_size)
        self.attn = nn.Linear(self.hidden_size * 2, self.max_length)
        self.attn_combine = nn.Linear(self.hidden_size * 2, self.hidden_size)
        self.dropout = nn.Dropout(self.dropout_p)
        self.gru = nn.GRU(self.hidden_size, self.hidden_size)
        self.out = nn.Linear(self.hidden_size, self.output_size)

    def forward(self, input_, hidden, encoder_outputs):
        embedded = self.embedding(input_).view(1, 1, -1)
        embedded = self.dropout(embedded)

        attn_weights = F.softmax(
            self.attn(torch.cat((embedded[0], hidden[0]), 1)), dim=1)
        attn_applied = torch.bmm(attn_weights.unsqueeze(0),
                                 encoder_outputs.unsqueeze(0))

        output = torch.cat((embedded[0], attn_applied[0]), 1)
        output = self.attn_combine(output).unsqueeze(0)

        #output = F.relu(output)
        output, hidden = self.gru(output, hidden)

        output = F.log_softmax(self.out(output[0]), dim=1)
        return output, hidden, attn_weights

    def initHidden(self):
        return torch.zeros(1, 1, self.hidden_size)

In [8]:
def train(input_tensor, target_tensor, encoder, decoder, encoder_optimizer, decoder_optimizer, criterion, max_length=10):
    encoder_hidden = encoder.initHidden()

    encoder_optimizer.zero_grad()
    decoder_optimizer.zero_grad()

    input_length = input_tensor.size(0)
    target_length = target_tensor.size(0)
    
    encoder_outputs = torch.zeros(max_length, encoder.hidden_size)

    loss = 0

    for ei in range(input_length):
        encoder_output, encoder_hidden = encoder(
            input_tensor[ei], encoder_hidden)
        encoder_outputs[ei] = encoder_output[0, 0]

    decoder_input = torch.tensor([[0]])

    decoder_hidden = encoder_hidden

    for di in range(target_length):
        decoder_output, decoder_hidden, decoder_attention = decoder(
            decoder_input, decoder_hidden, encoder_outputs)
        topv, topi = decoder_output.topk(1)
        decoder_input = topi.squeeze().detach()  # detach from history as input

        loss += criterion(decoder_output, target_tensor[di])
        if decoder_input.item() == 1:
            break

    loss.backward()

    encoder_optimizer.step()
    decoder_optimizer.step()

    return loss.item() / target_length

In [9]:
encoder = EncoderRNN(len(data_set.input_vocab2index)+2, 30)
attn_decoder1 = AttnDecoderRNN(30, len(data_set.output_vocab2index)+2, dropout_p=0.1)

encoder_optimizer = torch.optim.SGD(encoder.parameters(), lr=0.01)
decoder_optimizer = torch.optim.SGD(attn_decoder1.parameters(), lr=0.01)
criterion = nn.NLLLoss()

In [10]:
print_loss_total = 0
for epoch in range(num_epochs):
    for i, (data, target) in enumerate(data_loader, start=1):
        for j in range(len(data)):
            loss = train(data[j], target[j], encoder,
                         attn_decoder1, encoder_optimizer, decoder_optimizer, criterion)
            print_loss_total += loss
        
        print_loss_avg = print_loss_total / len(data_loader)
        print_loss_total = 0
        print('(%d %d%%) %.4f' % (i, i / len(data_loader) * 100, print_loss_avg))

(1 0%) 2.0879
(2 1%) 0.4850
(3 1%) 0.4591
(4 2%) 0.5571
(5 3%) 1.2201
(6 3%) 1.4394
(7 4%) 1.1418
(8 5%) 0.7183
(9 5%) 1.0668
(10 6%) 0.7943
(11 7%) 0.8301
(12 7%) 0.8559
(13 8%) 0.9772
(14 8%) 0.8835
(15 9%) 0.7920
(16 10%) 0.9752
(17 10%) 0.7000
(18 11%) 0.9448
(19 12%) 1.0604
(20 12%) 0.6881
(21 13%) 0.6406
(22 14%) 0.9323
(23 14%) 0.6754
(24 15%) 1.0985
(25 15%) 0.8704
(26 16%) 0.9467
(27 17%) 0.9277
(28 17%) 0.9834
(29 18%) 1.0656
(30 19%) 0.7809
(31 19%) 0.9251
(32 20%) 0.8919
(33 21%) 0.7152
(34 21%) 0.9149
(35 22%) 0.7019
(36 22%) 0.7909
(37 23%) 0.8838
(38 24%) 0.9017
(39 24%) 0.8554
(40 25%) 0.8923
(41 26%) 0.8310
(42 26%) 1.2580
(43 27%) 1.0548
(44 28%) 1.0098
(45 28%) 1.0442
(46 29%) 1.0065
(47 29%) 1.0325
(48 30%) 1.1922
(49 31%) 0.9035
(50 31%) 0.9370
(51 32%) 0.8973
(52 33%) 0.6977
(53 33%) 1.0527
(54 34%) 0.9095
(55 35%) 1.0727
(56 35%) 0.9101
(57 36%) 1.0087
(58 36%) 0.9463
(59 37%) 0.8609
(60 38%) 0.9975
(61 38%) 0.8148
(62 39%) 1.2787
(63 40%) 1.2601
(64 40%) 1.0627


(37 23%) 0.9362
(38 24%) 0.8976
(39 24%) 0.8371
(40 25%) 0.7503
(41 26%) 0.7661
(42 26%) 0.8787
(43 27%) 0.8313
(44 28%) 0.6141
(45 28%) 0.8220
(46 29%) 0.8934
(47 29%) 0.8535
(48 30%) 0.9034
(49 31%) 0.8698
(50 31%) 0.8590
(51 32%) 0.8210
(52 33%) 0.7497
(53 33%) 0.8183
(54 34%) 0.9706
(55 35%) 0.7787
(56 35%) 0.9368
(57 36%) 0.6734
(58 36%) 0.9086
(59 37%) 0.7396
(60 38%) 0.6879
(61 38%) 0.9149
(62 39%) 0.7347
(63 40%) 0.8468
(64 40%) 0.7560
(65 41%) 0.8292
(66 42%) 0.7876
(67 42%) 0.7826
(68 43%) 0.6829
(69 43%) 0.8262
(70 44%) 0.9926
(71 45%) 0.7932
(72 45%) 0.7424
(73 46%) 0.8610
(74 47%) 0.9582
(75 47%) 0.7667
(76 48%) 0.9153
(77 49%) 0.7335
(78 49%) 0.5891
(79 50%) 0.8571
(80 50%) 0.6773
(81 51%) 0.8087
(82 52%) 0.7619
(83 52%) 0.7671
(84 53%) 0.9567
(85 54%) 0.7483
(86 54%) 0.7487
(87 55%) 0.7633
(88 56%) 0.7353
(89 56%) 0.8241
(90 57%) 0.8112
(91 57%) 0.8364
(92 58%) 0.7545
(93 59%) 0.9074
(94 59%) 0.8599
(95 60%) 0.8749
(96 61%) 0.7852
(97 61%) 0.8586
(98 62%) 0.7622
(99 63%)

(72 45%) 0.6558
(73 46%) 0.7250
(74 47%) 0.7118
(75 47%) 0.6438
(76 48%) 0.7552
(77 49%) 0.8036
(78 49%) 0.6962
(79 50%) 0.7099
(80 50%) 0.5993
(81 51%) 0.7074
(82 52%) 0.7739
(83 52%) 0.7697
(84 53%) 0.8237
(85 54%) 0.7821
(86 54%) 0.7989
(87 55%) 0.8711
(88 56%) 0.7195
(89 56%) 0.7686
(90 57%) 0.8605
(91 57%) 0.7430
(92 58%) 0.7321
(93 59%) 0.7496
(94 59%) 0.8201
(95 60%) 0.7552
(96 61%) 0.6300
(97 61%) 0.8704
(98 62%) 0.7494
(99 63%) 0.8285
(100 63%) 0.6654
(101 64%) 0.5399
(102 64%) 0.6938
(103 65%) 0.6747
(104 66%) 0.7810
(105 66%) 0.5425
(106 67%) 0.8710
(107 68%) 0.7329
(108 68%) 0.5969
(109 69%) 0.6867
(110 70%) 0.7463
(111 70%) 0.8006
(112 71%) 0.6993
(113 71%) 0.7112
(114 72%) 0.7568
(115 73%) 0.7906
(116 73%) 0.6550
(117 74%) 0.6590
(118 75%) 0.7167
(119 75%) 0.6337
(120 76%) 0.6478
(121 77%) 0.6737
(122 77%) 0.7921
(123 78%) 0.7217
(124 78%) 0.8272
(125 79%) 0.7256
(126 80%) 0.5205
(127 80%) 0.6354
(128 81%) 0.6565
(129 82%) 0.7071
(130 82%) 0.7334
(131 83%) 0.6350
(132 84%

(107 68%) 0.8126
(108 68%) 0.6819
(109 69%) 0.4826
(110 70%) 0.6280
(111 70%) 0.7520
(112 71%) 0.7684
(113 71%) 0.7448
(114 72%) 0.7080
(115 73%) 0.7073
(116 73%) 0.6346
(117 74%) 0.7080
(118 75%) 0.7070
(119 75%) 0.7929
(120 76%) 0.6846
(121 77%) 0.7379
(122 77%) 0.6272
(123 78%) 0.7237
(124 78%) 0.7128
(125 79%) 0.7390
(126 80%) 0.6645
(127 80%) 0.6835
(128 81%) 0.6651
(129 82%) 0.6903
(130 82%) 0.6816
(131 83%) 0.7553
(132 84%) 0.5679
(133 84%) 0.6279
(134 85%) 0.6202
(135 85%) 0.5886
(136 86%) 0.8009
(137 87%) 0.6371
(138 87%) 0.6166
(139 88%) 0.6854
(140 89%) 0.7195
(141 89%) 0.7273
(142 90%) 0.6672
(143 91%) 0.6373
(144 91%) 0.5797
(145 92%) 0.6480
(146 92%) 0.6169
(147 93%) 0.7308
(148 94%) 0.7062
(149 94%) 0.5442
(150 95%) 0.6070
(151 96%) 0.6610
(152 96%) 0.6746
(153 97%) 0.6028
(154 98%) 0.6035
(155 98%) 0.7449
(156 99%) 0.7490
(157 100%) 0.2401
(1 0%) 0.6157
(2 1%) 0.7383
(3 1%) 0.6352
(4 2%) 0.6732
(5 3%) 0.6824
(6 3%) 0.7249
(7 4%) 0.7599
(8 5%) 0.7244
(9 5%) 0.6035
(10 6%

(140 89%) 0.5813
(141 89%) 0.5960
(142 90%) 0.7167
(143 91%) 0.6428
(144 91%) 0.6316
(145 92%) 0.6230
(146 92%) 0.5097
(147 93%) 0.6685
(148 94%) 0.6670
(149 94%) 0.7002
(150 95%) 0.5584
(151 96%) 0.6197
(152 96%) 0.7053
(153 97%) 0.6884
(154 98%) 0.5053
(155 98%) 0.5534
(156 99%) 0.6598
(157 100%) 0.1779
(1 0%) 0.6132
(2 1%) 0.6581
(3 1%) 0.6524
(4 2%) 0.5331
(5 3%) 0.6555
(6 3%) 0.6245
(7 4%) 0.5569
(8 5%) 0.5203
(9 5%) 0.6303
(10 6%) 0.6049
(11 7%) 0.4509
(12 7%) 0.5480
(13 8%) 0.6109
(14 8%) 0.5989
(15 9%) 0.6061
(16 10%) 0.5904
(17 10%) 0.5470
(18 11%) 0.6236
(19 12%) 0.6814
(20 12%) 0.6538
(21 13%) 0.5535
(22 14%) 0.6318
(23 14%) 0.5432
(24 15%) 0.6936
(25 15%) 0.6975
(26 16%) 0.6698
(27 17%) 0.5358
(28 17%) 0.5799
(29 18%) 0.5929
(30 19%) 0.8401
(31 19%) 0.9316
(32 20%) 0.5494
(33 21%) 0.7136
(34 21%) 0.6922
(35 22%) 0.7106
(36 22%) 0.7032
(37 23%) 0.6235
(38 24%) 0.5689
(39 24%) 0.6438
(40 25%) 0.6853
(41 26%) 0.6250
(42 26%) 0.5829
(43 27%) 0.7041
(44 28%) 0.7024
(45 28%) 0.63

(18 11%) 0.6248
(19 12%) 0.6534
(20 12%) 0.6650
(21 13%) 0.4927
(22 14%) 0.5852
(23 14%) 0.6906
(24 15%) 0.5669
(25 15%) 0.5691
(26 16%) 0.7139
(27 17%) 0.6030
(28 17%) 0.6432
(29 18%) 0.5340
(30 19%) 0.5449
(31 19%) 0.6040
(32 20%) 0.7585
(33 21%) 0.6928
(34 21%) 0.6749
(35 22%) 0.6815
(36 22%) 0.6287
(37 23%) 0.6015
(38 24%) 0.6846
(39 24%) 0.4881
(40 25%) 0.6613
(41 26%) 0.6547
(42 26%) 0.5464
(43 27%) 0.6047
(44 28%) 0.5032
(45 28%) 0.6410
(46 29%) 0.5556
(47 29%) 0.5471
(48 30%) 0.6315
(49 31%) 0.6642
(50 31%) 0.5644
(51 32%) 0.6232
(52 33%) 0.6443
(53 33%) 0.5049
(54 34%) 0.6345
(55 35%) 0.6020
(56 35%) 0.6429
(57 36%) 0.3959
(58 36%) 0.5685
(59 37%) 0.5980
(60 38%) 0.5538
(61 38%) 0.5850
(62 39%) 0.5774
(63 40%) 0.6282
(64 40%) 0.5813
(65 41%) 0.8435
(66 42%) 0.5851
(67 42%) 0.5639
(68 43%) 0.6813
(69 43%) 0.6775
(70 44%) 0.5976
(71 45%) 0.6157
(72 45%) 0.6202
(73 46%) 0.6264
(74 47%) 0.6088
(75 47%) 0.5496
(76 48%) 0.6002
(77 49%) 0.5319
(78 49%) 0.6596
(79 50%) 0.6424
(80 50%)

(53 33%) 0.5536
(54 34%) 0.7339
(55 35%) 0.4377
(56 35%) 0.6020
(57 36%) 0.5549
(58 36%) 0.4789
(59 37%) 0.5439
(60 38%) 0.6232
(61 38%) 0.6402
(62 39%) 0.7018
(63 40%) 0.7098
(64 40%) 0.7613
(65 41%) 0.5668
(66 42%) 0.7708
(67 42%) 0.6733
(68 43%) 0.6453
(69 43%) 0.6608
(70 44%) 0.6609
(71 45%) 0.5665
(72 45%) 0.6051
(73 46%) 0.6676
(74 47%) 0.6507
(75 47%) 0.6452
(76 48%) 0.5098
(77 49%) 0.7749
(78 49%) 0.5946
(79 50%) 0.6465
(80 50%) 0.5751
(81 51%) 0.7855
(82 52%) 0.4956
(83 52%) 0.5909
(84 53%) 0.5289
(85 54%) 0.5876
(86 54%) 0.6558
(87 55%) 0.5215
(88 56%) 0.5887
(89 56%) 0.5465
(90 57%) 0.5526
(91 57%) 0.7971
(92 58%) 0.5905
(93 59%) 0.6589
(94 59%) 0.5677
(95 60%) 0.5638
(96 61%) 0.5513
(97 61%) 0.5819
(98 62%) 0.6539
(99 63%) 0.4963
(100 63%) 0.7961
(101 64%) 0.6360
(102 64%) 0.5404
(103 65%) 0.5874
(104 66%) 0.7523
(105 66%) 0.5279
(106 67%) 0.5424
(107 68%) 0.4861
(108 68%) 0.7547
(109 69%) 0.6219
(110 70%) 0.6143
(111 70%) 0.6685
(112 71%) 0.6055
(113 71%) 0.7189
(114 72%) 

(88 56%) 0.7216
(89 56%) 0.5285
(90 57%) 0.5493
(91 57%) 0.6159
(92 58%) 0.6879
(93 59%) 0.6093
(94 59%) 0.7669
(95 60%) 0.6667
(96 61%) 0.5512
(97 61%) 0.6729
(98 62%) 0.5887
(99 63%) 0.5621
(100 63%) 0.5553
(101 64%) 0.6133
(102 64%) 0.6190
(103 65%) 0.5659
(104 66%) 0.5529
(105 66%) 0.5725
(106 67%) 0.5942
(107 68%) 0.6455
(108 68%) 0.6230
(109 69%) 0.5914
(110 70%) 0.6025
(111 70%) 0.7404
(112 71%) 0.5828
(113 71%) 0.5831
(114 72%) 0.7930
(115 73%) 0.6157
(116 73%) 0.6727
(117 74%) 0.5821
(118 75%) 0.5755
(119 75%) 0.5534
(120 76%) 0.6037
(121 77%) 0.6617
(122 77%) 0.6733
(123 78%) 0.6197
(124 78%) 0.6521
(125 79%) 0.6272
(126 80%) 0.4577
(127 80%) 0.4750
(128 81%) 0.5122
(129 82%) 0.4156
(130 82%) 0.8487
(131 83%) 0.6255
(132 84%) 0.6268
(133 84%) 0.5968
(134 85%) 0.5124
(135 85%) 0.5607
(136 86%) 0.6489
(137 87%) 0.5349
(138 87%) 0.4946
(139 88%) 0.5668
(140 89%) 0.6986
(141 89%) 0.5231
(142 90%) 0.6697
(143 91%) 0.7153
(144 91%) 0.7139
(145 92%) 0.6465
(146 92%) 0.6463
(147 93%)

(122 77%) 0.5317
(123 78%) 0.6566
(124 78%) 0.6307
(125 79%) 0.6571
(126 80%) 0.6332
(127 80%) 0.4810
(128 81%) 0.5998
(129 82%) 0.6157
(130 82%) 0.6589
(131 83%) 0.6851
(132 84%) 0.6455
(133 84%) 0.5341
(134 85%) 0.6572
(135 85%) 0.6247
(136 86%) 0.6082
(137 87%) 0.5339
(138 87%) 0.6765
(139 88%) 0.5987
(140 89%) 0.6016
(141 89%) 0.6086
(142 90%) 0.6632
(143 91%) 0.5800
(144 91%) 0.5620
(145 92%) 0.4848
(146 92%) 0.5627
(147 93%) 0.6677
(148 94%) 0.6906
(149 94%) 0.6353
(150 95%) 0.5385
(151 96%) 0.6290
(152 96%) 0.6568
(153 97%) 0.5248
(154 98%) 0.5838
(155 98%) 0.6941
(156 99%) 0.5713
(157 100%) 0.1371
(1 0%) 0.5552
(2 1%) 0.7006
(3 1%) 0.7045
(4 2%) 0.6183
(5 3%) 0.6004
(6 3%) 0.6405
(7 4%) 0.7307
(8 5%) 0.6468
(9 5%) 0.6517
(10 6%) 0.5601
(11 7%) 0.5405
(12 7%) 0.5486
(13 8%) 0.6654
(14 8%) 0.5454
(15 9%) 0.8158
(16 10%) 0.6274
(17 10%) 0.6516
(18 11%) 0.6644
(19 12%) 0.4966
(20 12%) 0.6851
(21 13%) 0.5529
(22 14%) 0.6072
(23 14%) 0.7061
(24 15%) 0.6223
(25 15%) 0.6669
(26 16%) 0.

(155 98%) 0.6276
(156 99%) 0.5824
(157 100%) 0.1679
(1 0%) 0.6005
(2 1%) 0.5753
(3 1%) 0.6195
(4 2%) 0.6266
(5 3%) 0.5102
(6 3%) 0.5810
(7 4%) 0.6820
(8 5%) 0.6433
(9 5%) 0.5782
(10 6%) 0.5051
(11 7%) 0.5294
(12 7%) 0.6827
(13 8%) 0.6983
(14 8%) 0.6271
(15 9%) 0.4835
(16 10%) 0.6167
(17 10%) 0.5318
(18 11%) 0.5712
(19 12%) 0.6496
(20 12%) 0.6719
(21 13%) 0.6117
(22 14%) 0.5696
(23 14%) 0.5605
(24 15%) 0.5562
(25 15%) 0.5730
(26 16%) 0.4904
(27 17%) 0.5429
(28 17%) 0.6018
(29 18%) 0.6231
(30 19%) 0.5837
(31 19%) 0.6627
(32 20%) 0.5628
(33 21%) 0.7217
(34 21%) 0.5343
(35 22%) 0.5388
(36 22%) 0.6183
(37 23%) 0.6178
(38 24%) 0.5264
(39 24%) 0.5595
(40 25%) 0.7604
(41 26%) 0.6436
(42 26%) 0.5522
(43 27%) 0.4903
(44 28%) 0.6494
(45 28%) 0.7480
(46 29%) 0.7051
(47 29%) 0.8389
(48 30%) 0.6667
(49 31%) 0.6834
(50 31%) 0.7645
(51 32%) 0.5202
(52 33%) 0.5984
(53 33%) 0.6511
(54 34%) 0.5528
(55 35%) 0.6830
(56 35%) 0.5203
(57 36%) 0.5019
(58 36%) 0.6816
(59 37%) 0.8104
(60 38%) 0.8788
(61 38%) 0.6